Run the imports and then the creation of the ep_names list.

At present can skip original data cleaning: now just adding updates between the last two save-points. Go to _Secondary name conversions_ to define the functions, then run the two cells in _Application_.

In [2]:
from tools import ova_combo
from tools import aifdb_fetcher
from src import xaif_toolbox
import pandas as pd 
import regex as re 
import os
import json
import urllib
import csv
import wget
import importlib
import requests
from glob import glob

In [4]:
with open('./tests/qt30_test/qt_ep_true_shortnames.csv', 'r', encoding='utf-8-sig') as f:
    qt30_ep_tuples = [tuple(line) for line in csv.reader(f)]
ep_names = [t[1] for t in qt30_ep_tuples]

In [4]:
importlib.reload(ova_combo)

<module 'tools.ova_combo' from '/Users/eimear/GitHub/papa/tools/ova_combo.py'>

Some of the nodesets reported to be part of the corpus have no OVA files. Some of the texts have no timestamps in them.

## Download QT30 OVA by episode

In [2]:
url = f"http://corpora.aifdb.org/nodesets.php?shortname=qt30"
with urllib.request.urlopen(url) as url:
    nodesets = json.loads(url.read().decode())

In [3]:
with open('./tests/qt30_test/qt30_shortcorps.txt', 'r') as file:
        qt30_episode_list = file.read().splitlines()

Check the nodes that are listed for each shortname in the list

In [9]:
base_dir = "./self_test/orig_episode_maps"
map_type = 'ova'

for shortname, epname in qt30_ep_tuples:
    dir_out = f"{base_dir}/{epname}"
    #os.path.join(base_dir, epname)

    if not os.path.exists(dir_out):
        os.makedirs(dir_out)


    url = f"http://corpora.aifdb.org/nodesets.php?shortname={shortname}"
    with urllib.request.urlopen(url) as url:
        data = json.loads(url.read().decode())
    map_list = data['nodeSets']
    print(f"\nNodesets for {epname}:", data['nodeSets'])
    missing = []
    
    for i, n in enumerate(data['nodeSets']):
        print("----")
        print(f"Checking for argmap at nodeset {n} (node {i+1}/{len(data['nodeSets'])})")
        filepath = f"{dir_out}/{str(n)}.json" #os.path.join(dir_out, f"{str(n)}.json")

        # print(f"Checking for file {filepath}")
        if os.path.exists(filepath):
            pass
            # print("Already downloaded!")
        else:
            # print("File doesn't exist yet.")
            # fileurl = url_base + n
            if map_type == 'ova':
                fileurl = f"http://ova.arg.tech/db/{n}"
                # print(f"\nGetting OVA-friendly file", end='')
            else:
                fileurl = f"http://www.aifdb.org/json/{n}"
                # print(f"\nGetting json file at {fileurl}",  end='')

            print(f" Getting url {fileurl}... ")
            try:
                wget.download(fileurl, filepath)
                print("\nDone")
            except urllib.error.HTTPError:
                missing.append(n)
                print(f"404: Nodeset {n} appears not to exist")
print("----")
# print(f"Finished with the argument maps in {epname}")
print("==========")


Nodesets for 11november2021: [23460, 23473, 23475, 23476, 23478, 23479, 23480, 23481, 23483, 23484, 23485, 23487, 23488, 23491, 23494, 23495, 23496, 23497, 23498, 23500, 23502, 23503, 23504, 23505, 23506, 23507, 23508, 23509, 23510, 23511, 23512, 23513, 23514, 23515, 23517, 23555, 23688, 24042, 25475, 25409, 25690, 25691]
----
Checking for argmap at nodeset 23460 (node 1/42)
 Getting url http://ova.arg.tech/db/23460... 


KeyboardInterrupt: 

(Server v slow when re-doing the above, so instead of waiting just cancelled and took from existing download.)

Reporting on missing OVA files

In [32]:
missing_nodesets = pd.DataFrame(columns=['shortname', 'episode', 'missing ova nodeset'])

In [33]:
for shortname, epname in qt30_ep_tuples:
    url = f"http://corpora.aifdb.org/nodesets.php?shortname={shortname}"
    with urllib.request.urlopen(url) as url:
        data = json.loads(url.read().decode())
    map_list = data['nodeSets']
    print(f"Nodesets for {epname}:", data['nodeSets'])

    for n in data['nodeSets']:
        fileurl = f"http://ova.arg.tech/db/{n}"
        r = requests.head(fileurl)
        if r.status_code != 200:
            missing_nodesets = missing_nodesets._append({'shortname': shortname, 'episode': epname, 'missing ova nodeset': n}, ignore_index=True)


Nodesets for 11november2021: [23460, 23473, 23475, 23476, 23478, 23479, 23480, 23481, 23483, 23484, 23485, 23487, 23488, 23491, 23494, 23495, 23496, 23497, 23498, 23500, 23502, 23503, 23504, 23505, 23506, 23507, 23508, 23509, 23510, 23511, 23512, 23513, 23514, 23515, 23517, 23555, 23688, 24042, 25475, 25409, 25690, 25691]
Nodesets for 28october2021: [23521, 23523, 23525, 23526, 23527, 23530, 23531, 23533, 23535, 23536, 23537, 23538, 23539, 23541, 23542, 23544, 23545, 23547, 23548, 23549, 23550, 23551, 23552, 23553, 23556, 23557, 23558, 23559, 23560, 23561, 23562, 23563, 23565, 23566, 23567, 23569, 23570, 23571, 23572, 23573, 23574, 23575, 23576, 23577, 23578, 23579, 23580, 23581, 23582, 23583, 23584, 23585, 23586, 23587, 23588, 25379, 25474, 23568, 25692]
Nodesets for 14october2021: [23589, 23590, 23591, 23593, 23594, 23595, 23596, 23597, 23599, 23600, 23601, 23602, 23603, 23604, 23605, 23606, 23607, 23608, 23609, 23610, 23611, 23612, 23613, 23614, 23615, 23616, 23617, 23618, 23620, 23

In [31]:
missing_nodesets

,shortname,episode,missing ova nodeset
0,qt11112021wtxt,11november2021,25475
1,qt28102021wtxt,28october2021,25474
2,qt14102021wtxt,14october2021,25472
3,qt30092021wtxt,30september2021,25045
4,qt16092021wtxt,16september2021,25473
5,qt02092021wtxt,2september2021,25468
6,qt19082021wtxt,19august2021,25465
7,qt05082021wtxt,5august2021,24992
8,qt22072021wtxt,22july2021,25463
9,cutietestrun8july2021,8july2021,25462


Quite a few appear to have no OVA file...

## Data Cleaning

Original name fix

### Original

In [4]:
def qt30_text_namefix(text_to_fix, filename):
    for fix_it in [('Luke McCroy Jones', 'Luke McCroy-Jones'), ('Kate Frobes', 'Kate Forbes'),
                   ('Chris Philip', 'Chris Philp'), ('Goerge Mpanga', 'George Mpanga'), ('George MPanga', 'George Mpanga')]:
        if fix_it[0] in text_to_fix:
            print(f"{os.path.basename(filename)}: {fix_it[0]} error")
            # text_to_fix = text_to_fix.replace(fix_it[0], fix_it[1])
            text_to_fix = re.sub(fix_it[0], fix_it[1], text_to_fix)
    return text_to_fix

In [11]:
for ep in ep_names:
    ova_combo.ova_all_ova3(f'./self_test/orig_episode_maps/{ep}', f'./self_test/ova3_episode_maps/{ep}')

./self_test/ova3_episode_maps/11november2021/23515.json
./self_test/ova3_episode_maps/11november2021/23503.json
./self_test/ova3_episode_maps/11november2021/23485.json
./self_test/ova3_episode_maps/11november2021/23688.json
./self_test/ova3_episode_maps/11november2021/23488.json
./self_test/ova3_episode_maps/11november2021/23484.json
./self_test/ova3_episode_maps/11november2021/23555.json
./self_test/ova3_episode_maps/11november2021/23502.json
./self_test/ova3_episode_maps/11november2021/23514.json
./self_test/ova3_episode_maps/11november2021/23509.json
./self_test/ova3_episode_maps/11november2021/23460.json
./self_test/ova3_episode_maps/11november2021/23476.json
./self_test/ova3_episode_maps/11november2021/23513.json
./self_test/ova3_episode_maps/11november2021/23505.json
./self_test/ova3_episode_maps/11november2021/25691.json
already OVA3
./self_test/ova3_episode_maps/11november2021/23483.json
./self_test/ova3_episode_maps/11november2021/23495.json
./self_test/ova3_episode_maps/11nov

In [95]:
for ep in ep_names:
    print(f'\n*** {ep} ***')
    ep_files = glob(f'./self_test/ova3_episode_maps/{ep}/*.json')
    print(ep_files)
    for mapfile in ep_files:
        print('\t', mapfile)

        with open(mapfile, 'r') as f:
            argmap = json.loads(f.read())
        
        for n in argmap['AIF']['nodes']:
            n['text'] = qt30_text_namefix(n['text'], mapfile)
        argmap['text'] = qt30_text_namefix(argmap['text'], mapfile)

        with open(mapfile, 'w') as f:
            json.dump(argmap, f, indent=4)


*** 11november2021 ***
['./self_test/ova3_episode_maps/11november2021/23515.json', './self_test/ova3_episode_maps/11november2021/23503.json', './self_test/ova3_episode_maps/11november2021/23485.json', './self_test/ova3_episode_maps/11november2021/23688.json', './self_test/ova3_episode_maps/11november2021/23488.json', './self_test/ova3_episode_maps/11november2021/23484.json', './self_test/ova3_episode_maps/11november2021/23555.json', './self_test/ova3_episode_maps/11november2021/23502.json', './self_test/ova3_episode_maps/11november2021/23514.json', './self_test/ova3_episode_maps/11november2021/23509.json', './self_test/ova3_episode_maps/11november2021/23460.json', './self_test/ova3_episode_maps/11november2021/23476.json', './self_test/ova3_episode_maps/11november2021/23513.json', './self_test/ova3_episode_maps/11november2021/23505.json', './self_test/ova3_episode_maps/11november2021/25691.json', './self_test/ova3_episode_maps/11november2021/23483.json', './self_test/ova3_episode_maps/

## Secondary name conversions

Non-typo name wrangling. Mainly audience member name conversions.

1. Convert transcript speaker-attribution of Andrew RT Davies to Andrew Davies to match L-nodes.
2. Insert 'Unknown' surname to match audience members where relevant.

Meta nodes should be cut before this, otherwise end up finding the annotators as speakers.

In [15]:
for ep in ep_names:
    if not os.path.exists(f'./self_test/ova3_mini_name_edits/{ep}'):
        os.makedirs(f'./self_test/ova3_mini_name_edits/{ep}')

In [6]:
# For consistency, replace instances of 'Andrew RT Davies' in the transcript 
# and in L-nodes with 'Andrew Davies'
def andy_fix(xaif, verbose=False):
    in_l = False
    in_txt = False
    
    corrected_nodes = []
    # Convert L-node speakers
    for l in xaif['AIF']['nodes']:
        if l['type'] == 'L':
            if re.search('Andrew RT Davies[\s]*:', l['text']):
                l['text'] = re.sub('Andrew RT Davies[\s]*:', 'Andrew Davies :', l['text'])
                in_l = True
                if verbose:
                    print(f"Node {l['nodeID']}: {l['text']}")
        corrected_nodes.append(l)
    xaif['AIF']['nodes'] = corrected_nodes
    
    # Convert transcript attributions
    if 'Andrew RT Davies' in xaif['text']:
        if verbose:
            print('\tAndrew RT Davies found in text')
        xaif['text'] = re.sub('Andrew RT Davies', 'Andrew Davies', xaif['text'])
        # xaif['text'] = re.sub(r"(^|(?<=[\.\"?\'!’…;>]))(Andrew RT Davies)(?=(<.*>)*\[.*?)", '\\1Andrew Davies\\3', xaif['text'])
    
    if verbose:
        if in_l:
            print('\tAndrew RT Davies found in L-node(s)')
    
    return xaif

def misc_name_fixes(xaif):
    for l in [n for n in xaif['AIF']['nodes'] if n['type'] == 'L']:
        l['text'] = re.sub('Craig Davis :', 'Craig Unknown :', l['text'])
    return xaif

# Make use of 'Unknown' consistent across nodes and transcript
# Assumes speaker attribution is identifiable by a space after speaker name.
def audience_unk(xaif, verbose=False):
    # if re.match("Unknown :", "Katie Unknown : I've not been able to see her since then"):
    
    # Check L-nodes
    corrected_nodes = []
    nodes_with_unk = []
    unk_spkrs = []
    # Make sure capitalisation of L-node speakers is consistent
    for n in xaif['AIF']['nodes']:
        if n['type'] != 'L':
            corrected_nodes.append(n)
        else:
            # Check for start to skip annotator nodes
            if re.findall("^[\w*\d*]+ [Uu]nknown[\s]*:", n['text']):
                n['text'] = re.sub('[Uu]nknown[\s]*:', 'Unknown :', n['text'])
                corrected_nodes.append(n)
                nodes_with_unk.append(n)

                splits = n['text'].split(':')
                if len(splits) < 2:
                    print(f"L-node with no recognisable speaker:\t{n['nodeID']}")
                else:
                    spkr = splits[0].strip()
                if spkr not in unk_spkrs:
                    unk_spkrs.append(spkr)
            else:
                corrected_nodes.append(n)
    if verbose:
        if len(unk_spkrs) > 0:
            print(unk_spkrs)
            print(xaif['text'])
    # Handle text
    for spkr in unk_spkrs:
        first_name = spkr.split()[0]
        # Allow for either immediate timestamp or immediate tag.
        xaif['text'] = re.sub(f"{first_name}[\s]*\[", f"{first_name} Unknown[", xaif['text'])
        xaif['text'] = re.sub(f"{first_name}[\s]*\<", f"{first_name} Unknown<", xaif['text'])

    xaif['AIF']['nodes'] = corrected_nodes
    return xaif

def audience_name2code(xaif, verbose=False):
    codename_nodes = []

    # Collect all L-nodes that have an AudienceMember speaker and are linked to the text
    l_nodes = [n for n in xaif['AIF']['nodes'] if n['type'] == 'L']
    if verbose:
        print(f"{len(l_nodes)} L-nodes to check")
    
    for l in l_nodes:
        codename = re.findall("^AudienceMember \d\d\d\d\d\d\d\dQT\d\d", l['text'])
        if codename:
            if verbose:
                print(f"\tFound {codename[0]} in node {l['nodeID']}, ", end='')
                # print(f"\t\t" + l['text'])
            direct_loc = re.findall(f"node{l['nodeID']}", xaif['text'])
            if direct_loc:
                if verbose:
                    print(f"node {l['nodeID']} found in text")
                codename_nodes.append({'name': codename[0], 'nodeID': l['nodeID']})
                # if verbose:
                    # print(codename_nodes[-1])
            else:
                if verbose:
                    print(f"node {l['nodeID']} not found in text")
            if verbose:
                print(f"\t{l['text']}")
                print()

    if verbose and len(codename_nodes) > 0:
        print("Now checking for each codname entry")
    # Check how the speaker is named in the transcript: if stripped version isn't codename, replace
    for n in codename_nodes:
        if verbose:
            print("Checking for node ", n['nodeID'])
        match_iter = re.finditer(f"(^|(?<=[\.\"?\'!’…;>]))([\'\s\w+]+)?(?=(<.*>)*\[.*?id=\"node{n['nodeID']})", xaif['text'])
        match_list = [m for m in match_iter]
        if verbose:
            print(f"\nList of matches for {n['nodeID']}")
            print(match_list)
        if len(match_list) == 0:
            print(f"Failed to match name {n['name']} for node {n['nodeID']}")
            continue
        else:
            # Orig picking last won't work if there's a linebreak between the name and stamp
            target = match_list[-1]
            # Change: work backwards skipping blanks until first non-blank
            for t in reversed(match_list):
                if t.group().strip() != '':
                    target = t
                    break
            if verbose:
                print(f"Matching ", target)
            if target.group().strip() != n['name'].strip():
                if verbose:
                    print('MISMATCH! Going to correct text')
                    print(f"{target.group().strip()} != {n['name'].strip()}")
            # print(xaif['text'][:target.span()[0]] + n['name'] + xaif['text'][target.span()[1]:])
                xaif['text'] = xaif['text'][:target.span()[0]] + n['name'] + xaif['text'][target.span()[1]:]
            else:
                print(f"Already correct for node {n['nodeID']}")
    return xaif

# Application

In ova3_episode_maps, 21326, 18878, 23808 content has been replaced by a copy with structural fixes that were too much hassle to. Argument anchoring inserted, bad reported extra nodes removed.

Hack fixes

In [13]:
space = 'hi    there'
re.sub(r'hi[\s]+', 'hi!', space)

'hi!there'

In [34]:
for i, ep in enumerate(ep_names):
    print(f"\n=== Episode {ep} ({i+1}/30)===")
    maps  = glob(f'./self_test/ova3_episode_maps/{ep}/*.json')
    if not os.path.exists(f'./self_test/ova3_mini_name_edits/{ep}'):
        os.makedirs(f'./self_test/ova3_mini_name_edits/{ep}')

    for mapfile in maps:
        print(os.path.basename(mapfile))
        with open(mapfile) as f:
            xaif = json.loads(f.read())
        # xaif = xaif_toolbox.remove_all_meta(xaif)

        xaif['text'] = re.sub('\[DONE\]', '', xaif['text'])

        # Hacky individual fixes for dodgy maps    
        # Main text fixes
        if os.path.basename(mapfile) == '20880.json':
            xaif['text'] = re.sub('Andrew Unknown', 'Andrew Davies', xaif['text'])
        elif os.path.basename(mapfile) == '18798.json':
            faulty_node = [n for n in xaif['AIF']['nodes'] if n['nodeID'] == "125_18798"][0]
            faulty_node.update({'text' : re.sub('Fiona Bruce', 'George Mpanga', faulty_node['text'])})
        elif os.path.basename(mapfile) == '25556.json':
            xaif['text'] = re.sub('\[DONE\]' ,'', xaif['text'])
        elif os.path.basename(mapfile) == '23603.json':
            xaif['AIF']['nodes'] == [n for n in xaif['AIF']['nodes'] if n['text'] != "he said nothing"]
            xaif['text'] = "Rosie Jones [0:15:19]" + xaif['text']
        elif os.path.basename(mapfile) == '23594.json':
            xaif['text'] = "Minette Batters [0:13:30]" + xaif['text']
        elif os.path.basename(mapfile) == '22954.json':
            xaif['text'] = "Fiona Bruce" + xaif['text']
        elif os.path.basename(mapfile) == '19757.json':
            xaif['text'] = re.sub('\[Sue\]', 'Sue Unknown[00:30:56]', xaif['text'])
            xaif['text'] = re.sub('\[Dave\]', 'Dave Unknown[00:31:07]', xaif['text'])
            xaif['text'] = re.sub('\[DONE\]', '', xaif['text'])




        # Node text fixes
        elif os.path.basename(mapfile) == '18760.json':
            faulty_nodes = [n for n in xaif['AIF']['nodes'] if n['type'] == 'L']
            for n in faulty_nodes:
                n.update({'text': re.sub('Female Audience[\s]*Member1', 'Kirsty Unknown', n['text'])})
                print('updating')
        elif os.path.basename(mapfile) == '18812.json':
            faulty_node = [n for n in xaif['AIF']['nodes'] if n['nodeID'] == "171_18812"][0]
            faulty_node.update({'text' : re.sub('Female Audience[\s]*Member', 'Emma Stephenson', faulty_node['text'])})
            print(faulty_node)
        elif os.path.basename(mapfile) == '25554.json':
            faulty_nodes = [n for n in xaif['AIF']['nodes'] if 'Unknown Unknown' in n['text']]
            for n in faulty_nodes:
                n.update({'text' : re.sub('Unknown Unknown', 'Unknown Speaker', n['text'])})
        elif os.path.basename(mapfile) == '21270.json':
            xaif['text'] = re.sub('Lias ', 'Lisa ', xaif['text'])
        elif os.path.basename(mapfile) == '21305.json':
            faulty_node = [n for n in xaif['AIF']['nodes'] if n['nodeID'] == "21_21305"][0]
            faulty_node['text'] = "Andi Unknown : " + faulty_node['text']
        elif os.path.basename(mapfile) == '21281.json':
            faulty_nodes = [n for n in xaif['AIF']['nodes'] if 'Tim Unknown' in n['text']]
            for n in faulty_nodes:
                n['text'] = re.sub('Tim Unknown', 'Tim Down', n['text'])
        elif os.path.basename(mapfile) == '18848.json':
            for n in xaif['AIF']['nodes']:
                if (n['type'] == 'L') and ('Berlinda' in n['text']):
                    n['text'] = re.sub('Berlinda', 'Belinda', n['text'])
        elif os.path.basename(mapfile) == '18849.json':
            for n in xaif['AIF']['nodes']:
                if (n['type'] == 'L') and ('Deborah Unknown' in n['text']):
                    n['text'] = re.sub('Deborah Unknown', 'Deborah Norrish', n['text'])
        elif os.path.basename(mapfile) == '23820.json':
            faulty_nodes = [n for n in xaif['AIF']['nodes'] if 'Mehdi Hasan' in n['text']]
            for n in faulty_nodes:
                n['text'] = re.sub('Mehdi Hasan', 'Nelufar Hedayat', n['text'])
        elif os.path.basename(mapfile) == '19737.json':
            faulty_nodes = [n for n in xaif['AIF']['nodes'] if 'Suan Desouza' in n['text']]
            for n in faulty_nodes:
                n['text'] = re.sub('Suan', 'Susan', n['text'])


        # Node type fixes
        elif os.path.basename(mapfile) == '18809.json':
            faulty_node = [n for n in xaif['AIF']['nodes'] if n['nodeID'] == '38_18809'][0]
            faulty_node.update({'type': 'I'})
        elif os.path.basename(mapfile) == '23607.json':
            faulty_node = [n for n in xaif['AIF']['nodes'] if n['nodeID'] == "209_23607"][0]
            faulty_node.update({'type': 'I'})
        elif os.path.basename(mapfile) == '21306.json':
            faulty_nodes = [n for n in xaif['AIF']['nodes'] if n['nodeID'] in ["125_21306", "127_21306"]]
            for n in faulty_nodes:
                n['text'] = "Asserting"
                n['type'] = "YA"

        # Wrong reported speech
        elif os.path.basename(mapfile) == '23154.json': 
            # flip edges
            faulty_edges = [e for e in xaif['AIF']['edges'] if e['edgeID'] in [10, 11, 71, 70, 77, 78]]
            for e in faulty_edges:
                e.update({'fromID': e['toID'], 'toID': e['fromID']})

            # flip types
            faulty_nodes = [n for n in xaif['AIF']['nodes'] if n['nodeID'] in ['40_23154', '36_23154', '146_23154', '83_23154', '123_23154', '117_23154']]
            for n in faulty_nodes:
                if n['type'] == 'L':
                    n.update({'type': 'I'})
                elif n['type'] == 'I':
                    n.update({'type': 'L'})
        
        elif os.path.basename(mapfile) == '23602.json': 
            # flip edges
            faulty_edges = [e for e in xaif['AIF']['edges'] if e['edgeID'] in [69, 68]]
            for e in faulty_edges:
                e.update({'fromID': e['toID'], 'toID': e['fromID']})

            # flip types
            faulty_nodes = [n for n in xaif['AIF']['nodes'] if n['nodeID'] in ['94_23602', '88_23602']]
            for n in faulty_nodes:
                if n['type'] == 'L':
                    n.update({'type': 'I'})
                elif n['type'] == 'I':
                    n.update({'type': 'L'})


        # Multiple fixes
        elif os.path.basename(mapfile) == '18790.json':
            faulty_nodes = [n for n in xaif['AIF']['nodes'] if 'Female Audience Member 3 :' in n['text']]
            for n in faulty_nodes:
                n.update({'text' : re.sub('Female Audience Member 3', 'Katie Unknown', n['text'])})
            xaif['text'] = re.sub('Katie\[', 'Katie Unknown[', xaif['text'])
        elif os.path.basename(mapfile) == '18866.json':
            faulty_nodes = [n for n in xaif['AIF']['nodes'] if 'Mike Maleaudience' in n['text']]
            for n in faulty_nodes:
                n.update({'text' : re.sub('Mike Maleaudience', 'Mike Unknown', n['text'])})
            xaif['text'] = re.sub('Mike\[', 'Mike Unknown[', xaif['text'])
        
        elif os.path.basename(mapfile) == '19756.json':
            # Fix text
            xaif['text'] = re.sub('\[Jim\]', 'Jim Unknown[00:29:52]', xaif['text'])
            xaif['text'] = re.sub('\[DONE\]', '', xaif['text'])
            xaif['text'] = re.sub('\[James\]', 'James Unknown[00:30:32]', xaif['text'])

            # Fix James nodes
            faulty_nodes = [n for n in xaif['AIF']['nodes'] if 'James ' in n['text']]
            for n in faulty_nodes:
                n.update({'text' : re.sub(r'James[\s]*:', 'James Unknown :', n['text'])})

            # Fix Jim nodes
            faulty_nodes = [n for n in xaif['AIF']['nodes'] if n['nodeID'] in ['40_19756', '47_19756', '54_19756', '69_19756', '80_19756']]
            for n in faulty_nodes:
                n.update({'text': re.sub('Fiona Bruce', 'Jim Unknown', n['text'])})


        elif os.path.basename(mapfile) == '21280.json':
            # Remove double name
            faulty_node = [n for n in xaif['AIF']['nodes'] if 'Tony Unkown : Tony Unkown :' in n['text']][0]
            faulty_node['text'] = re.sub('Tony Unkown : Tony Unkown :', 'Tony Unkown :', faulty_node['text'])
            
            # Fix L-nodes
            for n in [n for n in xaif['AIF']['nodes'] if n['type'] == 'L']:
                n.update({'text': re.sub('Tony Unkown', 'Tony Unknown', n['text'])})
        

        # Misc: duplicates
        elif os.path.basename(mapfile) == '23597.json':
            # Remove all duplicate nodes
            seen = []
            new_nodelist = []
            for node in xaif['AIF']['nodes']:
                t = tuple(node.items())
                if t not in seen:
                    seen.append(t)
                    new_nodelist.append(node)
            xaif['AIF']['nodes'] = new_nodelist

            # Remove all duplicate edges
            # only check to/from IDs, edge IDs of duplicates are unique
            seen = []
            new_edgelist = []
            for edge in xaif['AIF']['edges']:
                t = tuple([edge['fromID'], edge['toID']])
                if t not in seen:
                    seen.append(t)
                    new_edgelist.append(edge)
            xaif['AIF']['edges'] = new_edgelist

        elif os.path.basename(mapfile) == '19761.json':
            continue

        xaif = andy_fix(xaif)
        xaif = misc_name_fixes(xaif)
        xaif = audience_unk(xaif)
        with open(f'./self_test/ova3_mini_name_edits/{ep}/{os.path.basename(mapfile)}', 'w') as f:
            json.dump(xaif, f, indent=4)
        # print()
            
# Systematic fixes
for i, ep in enumerate(ep_names):
    print(f"\n=== Episode {ep} ({i+1}/30)===")
    maps  = glob(f'./self_test/ova3_mini_name_edits/{ep}/*.json')
    if not os.path.exists(f'./self_test/ova3_name_edits/{ep}'):
        os.makedirs(f'./self_test/ova3_name_edits/{ep}')

    for mapfile in maps:
        # print(os.path.basename(mapfile))
        print(f"Opening {mapfile}")
        with open(mapfile) as f:
            xaif = json.loads(f.read())
        # xaif = xaif_toolbox.remove_all_meta(xaif)
        xaif = audience_name2code(xaif, verbose=True)
        with open(f'./self_test/ova3_name_edits/{ep}/{os.path.basename(mapfile)}', 'w') as f:
            json.dump(xaif, f, indent=4)


=== Episode 11november2021 (1/30)===
23515.json
23503.json
23485.json
23688.json
23488.json
23484.json
23555.json
23502.json
23514.json
23509.json
23460.json
23476.json
23513.json
23505.json
25691.json
23483.json
23495.json
23494.json
25690.json
23504.json
24042.json
23512.json
25409.json
23498.json
23508.json
23497.json
23478.json
23481.json
23507.json
23511.json
23510.json
23506.json
23480.json
23496.json
23479.json
23475.json
23491.json
23487.json
23517.json
23473.json
23500.json

=== Episode 28october2021 (2/30)===
23542.json
23578.json
23581.json
23539.json
23558.json
23523.json
23574.json
23562.json
23535.json
23563.json
23575.json
23559.json
23538.json
23580.json
23579.json
23548.json
23525.json
23572.json
23533.json
23544.json
23552.json
23587.json
23568.json
23586.json
23569.json
23553.json
23545.json
23565.json
23573.json
23549.json
23531.json
23566.json
23570.json
23527.json
23585.json
23550.json
23547.json
23551.json
25692.json
23584.json
23526.json
23571.json
23567.json
2

Systematic fixes

In [29]:
for i, ep in enumerate(ep_names):
    print(f"\n=== Episode {ep} ({i+1}/30)===")
    maps  = glob(f'./self_test/ova3_mini_name_edits/{ep}/*.json')
    if not os.path.exists(f'./self_test/ova3_name_edits/{ep}'):
        os.makedirs(f'./self_test/ova3_name_edits/{ep}')

    for mapfile in maps:
        # print(os.path.basename(mapfile))
        print(f"Opening {mapfile}")
        with open(mapfile) as f:
            xaif = json.loads(f.read())
        # xaif = xaif_toolbox.remove_all_meta(xaif)
        xaif = audience_name2code(xaif, verbose=True)
        with open(f'./self_test/ova3_name_edits/{ep}/{os.path.basename(mapfile)}', 'w') as f:
            json.dump(xaif, f, indent=4)
    


=== Episode 11november2021 (1/30)===
Opening ./self_test/ova3_mini_name_edits/11november2021/23515.json
52 L-nodes to check
	Found AudienceMember 20211111QT18 in node 140_23515, node 140_23515 found in text
	AudienceMember 20211111QT18 : A lot of this comes to the cost of the person

	Found AudienceMember 20211111QT18 in node 147_23515, node 147_23515 found in text
	AudienceMember 20211111QT18 : Being in Eastleigh we're close to an airport and a station which have fantastic ability to get anywhere

	Found AudienceMember 20211111QT18 in node 155_23515, node 155_23515 found in text
	AudienceMember 20211111QT18 : I wanted to go to Edinburgh

	Found AudienceMember 20211111QT18 in node 166_23515, node 166_23515 found in text
	AudienceMember 20211111QT18 : I can't get there for £30 on a flight

	Found AudienceMember 20211111QT18 in node 188_23515, node 188_23515 found in text
	AudienceMember 20211111QT18 : I want to drive to be green

	Found AudienceMember 20211111QT18 in node 195_23515, no

# Make per-episode OVA

In [5]:
if not os.path.exists('./self_test/merged_episodes'):
    os.makedirs('./self_test/merged_episodes')
for ep in ep_names:
    # ova_combo.combine_ova3(f'./self_test/ova3_episode_maps/{ep}', f'./self_test/merged_episodes/{ep}.json', verbose=True)
    ova_combo.combine_ova3(f'./self_test/ova3_name_edits/{ep}', f'./self_test/merged_episodes/{ep}.json', verbose=True)


Combining files in the directory  ./self_test/ova3_name_edits/11november2021
41 files found.
Stamp list:  [('[00:51:44]', 0), ('[00:50:18]', 1), ('[00:17:54]', 2), ('[00:15:16]', 3), ('[00:39:14]', 4), ('[00:33:08]', 5), ('[00:11:24]', 6), ('[00:46:58]', 7), ('[01:04:40]', 8), ('[00:48:26]', 9), ('[00:10:36]', 10), ('[00:14:06]', 11), ('[01:01:10]', 12), ('[00:36:08]', 13), ('[00:59:22]', 14), ('[00:31:38]', 15), ('[00:30:20]', 16), ('[00:42:32]', 17), ('[00:56:04]', 18), ('[00:54:36]', 19), ('[00:34:50]', 20), ('[01:03:12]', 21), ('[00:11:56]', 22), ('[00:44:06]', 23), ('[00:53:14]', 24), ('[00:41:02]', 25), ('[00:20:56]', 26), ('[00:26:56]', 27), ('[00:57:34]', 28), ('[00:23:50]', 29), ('[01:06:36]', 30), ('[01:05:38]', 31), ('[00:22:20]', 32), ('[00:19:28]', 33), ('[00:28:46]', 34), ('[00:13:34]', 35), ('[00:16:20]', 36), ('[00:25:30]', 37), ('[00:45:20]', 38), ('[00:12:46]', 39), ('[00:37:48]', 40)]
Sorted stamp list:  [('[00:10:36]', 10), ('[00:11:24]', 6), ('[00:11:56]', 22), ('[

# Scratch

In [104]:
dict_list = [{
                "nodeID": "36_18809",
                "text": "Hyewon: Iona Bain : in the last week, we have seen a young couple in less centre banner from the front of their house saying buying this decision we ever made. \u00a0Come and ask us why",
                "type": "L"
            },
            {
                "nodeID": "38_18809",
                "text": "in the last week, we have seen a young couple hang a banner from the front of their house saying \"buying this was the worst decision we ever made, come and ask us why\"",
                "type": "L"
            },
            {
                "nodeID": "40_18809",
                "text": "Hyewon: young couple's banner : in the last week, we have seen a young couple in less centre banner from the front of their house saying buying this decision we ever made. \u00a0Come and ask us why",
                "type": "L"
            },
            {
                "nodeID": "42_18809",
                "text": "Asserting",
                "type": "YA"
            },
            {
                "nodeID": "43_18809",
                "text": "Iona Bain : their developer wasn't particularly happy about that",
                "type": "L"
            },
            {
                "nodeID": "44_18809",
                "text": "Asserting",
                "type": "YA"
            }]

In [105]:
faulty_node = [n for n in dict_list if n['nodeID'] == '38_18809'][0]
faulty_node.update({'type': 'I'})

In [106]:
dict_list

[{'nodeID': '36_18809',
  'text': 'Hyewon: Iona Bain : in the last week, we have seen a young couple in less centre banner from the front of their house saying buying this decision we ever made. \xa0Come and ask us why',
  'type': 'L'},
 {'nodeID': '38_18809',
  'text': 'in the last week, we have seen a young couple hang a banner from the front of their house saying "buying this was the worst decision we ever made, come and ask us why"',
  'type': 'I'},
 {'nodeID': '40_18809',
  'text': "Hyewon: young couple's banner : in the last week, we have seen a young couple in less centre banner from the front of their house saying buying this decision we ever made. \xa0Come and ask us why",
  'type': 'L'},
 {'nodeID': '42_18809', 'text': 'Asserting', 'type': 'YA'},
 {'nodeID': '43_18809',
  'text': "Iona Bain : their developer wasn't particularly happy about that",
  'type': 'L'},
 {'nodeID': '44_18809', 'text': 'Asserting', 'type': 'YA'}]